# Lightweight Fine-Tuning Project

TODO: In this cell, describe your choices for each of the following

* PEFT technique: Low-Rank Adaptation (LoRA)
* Model: bhadresh-savani/bert-base-uncased-emotion
* Evaluation approach: trainer.train() & trainer.evaluate()
* Fine-tuning dataset: Dair-ai/emotion dataset

## Loading and Evaluating a Foundation Model

TODO: In the cells below, load your chosen pre-trained Hugging Face model and evaluate its performance prior to fine-tuning. This step includes loading an appropriate tokenizer and dataset.

In [2]:
! pip install datasets --upgrade
! pip install --upgrade pyarrow
! pip install accelerate -U
! pip install transformers --upgrade
! pip install --upgrade peft

In [3]:
from datasets import load_dataset
dataset = load_dataset("dair-ai/emotion", trust_remote_code=True)

splits = ["train", "test"]
print(dataset["train"])
print(dataset["train"][0]["text"])
dataset

Dataset({
    features: ['text', 'label'],
    num_rows: 16000
})
i didnt feel humiliated


DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 16000
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
})

In [4]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(
    "bhadresh-savani/bert-base-uncased-emotion",
    num_labels=6,
    id2label={0: "sadness", 1: "joy", 2: "love", 3: "anger", 4: "fear", 5: "surprise"},
    label2id={"sadness":0, "joy":1, "love":2, "anger":3, "fear":4, "surprise":5},
)

for param in model.base_model.parameters():
    param.requires_grad = True

model.save_pretrained("initial-model")
model.classifier
model

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [5]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bhadresh-savani/bert-base-uncased-emotion")

def preprocess_function(input_data):
    return tokenizer(input_data["text"], truncation=True, padding=True)

tokenized_dataset = {}
for split in splits:
    tokenized_dataset[split] = dataset[split].map(preprocess_function, batched=True)

print(tokenized_dataset["train"])
print(tokenized_dataset["test"])
tokenized_dataset["train"][0]["input_ids"][:5]

Dataset({
    features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 16000
})
Dataset({
    features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 2000
})


[101, 1045, 2134, 2102, 2514]

In [6]:
import numpy as np
import torch
from transformers import DataCollatorWithPadding, Trainer, TrainingArguments


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return {"accuracy": (predictions == labels).mean()}


trainer = Trainer(
    model=model,
    args=TrainingArguments(
        output_dir="./project1/data/sentiment_classifier",
        learning_rate=2e-3,
        use_cpu=False,
        per_device_train_batch_size=8,
        per_device_eval_batch_size=8,
        evaluation_strategy="epoch",
        save_strategy="epoch",
        num_train_epochs=2,
        weight_decay=0.01,
        load_best_model_at_end=True,
    ),
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    compute_metrics=compute_metrics,
)


trainer.train()
eval_results_standard=trainer.evaluate()

/Users/ed/anaconda3/lib/python3.11/site-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy
1,1.735300,1.652069,0.290500
2,1.620600,1.561970,0.347500


## Performing Parameter-Efficient Fine-Tuning

TODO: In the cells below, create a PEFT model from your loaded model, run a training loop, and save the PEFT model weights.

In [5]:
from peft import LoraConfig, TaskType
loraConfig=LoraConfig(
    task_type=TaskType.SEQ_CLS,
    r=8,
    lora_alpha=32,
    lora_dropout=0.1,
    bias="none",
    target_modules=['query','value']
)

initial_model=model.from_pretrained("initial-model")

In [10]:
from peft import get_peft_model #, merge_and_unload

lora_model = get_peft_model(initial_model,loraConfig)
lora_model.print_trainable_parameters()
#Reduce the memory footprint by merging adapter weights with the model
lora_model = lora_model.merge_and_unload(initial_model,loraConfig)
loraConfig.target_modules

trainable params: 299,526 || all params: 109,786,380 || trainable%: 0.2728261921014246


Unloading and merging model: 100%|██████████| 258/258 [00:00<00:00, 3370.79it/s]


{'query', 'value'}

In [8]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return {"accuracy": (predictions == labels).mean()}

trainer_forPEFT = Trainer(
    model=lora_model,
    args=TrainingArguments(
        output_dir="./project1/data/sentiment_classifier_wpeft",
        learning_rate=2e-3,
        use_cpu=False,
        per_device_train_batch_size=4,
        per_device_eval_batch_size=4,
        evaluation_strategy="epoch",
        save_strategy="epoch",
        num_train_epochs=1,
        weight_decay=0.01,
        load_best_model_at_end=True,
    ),
    train_dataset=tokenized_dataset["train"].rename_column("label","labels"),
    eval_dataset=tokenized_dataset["test"].rename_column("label","labels"),
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    compute_metrics=compute_metrics,
)

trainer_forPEFT.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,1.601800,1.561532,0.347500


TrainOutput(global_step=4000, training_loss=1.4981998672485353, metrics={'train_runtime': 3686.9506, 'train_samples_per_second': 4.34, 'train_steps_per_second': 1.085, 'total_flos': 629726633071200.0, 'train_loss': 1.4981998672485353, 'epoch': 1.0})

In [ ]:
eval_results_wLoRA=trainer_forPEFT.evaluate()

In [ ]:
lora_model.save_pretrained("lora-model")


## Performing Inference with a PEFT Model

TODO: In the cells below, load the saved PEFT model weights and evaluate the performance of the trained PEFT model. Be sure to compare the results to the results from prior to fine-tuning.

In [14]:
tokenized_dataset_validation = {}
tokenized_dataset_validation["validation"] = dataset["validation"].map(preprocess_function, batched=True)

tokenized_dataset_validation

{'validation': Dataset({
     features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
     num_rows: 2000
 })}

In [23]:
import pandas as pd

predictions = trainer.predict(tokenized_dataset_validation["validation"])
predictions_wPEFT = trainer_forPEFT.predict(tokenized_dataset_validation["validation"])

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (3,) + inhomogeneous part.

In [27]:
predictions

PredictionOutput(predictions=array([[ 6.9475574 , -1.3199435 , -0.97556376, -1.0141709 , -1.562117  ,
        -1.7578284 ],
       [ 6.927617  , -1.294594  , -1.6009897 , -0.9777841 , -0.9601297 ,
        -1.4807993 ],
       [-1.7349471 ,  5.123596  ,  3.3889031 , -2.0377204 , -2.675888  ,
        -2.2464695 ],
       ...,
       [-1.2634884 ,  7.087826  , -0.7833872 , -1.7968845 , -2.3344681 ,
        -1.3011024 ],
       [-2.1280227 ,  5.2574053 ,  2.3886573 , -1.3714836 , -3.0220609 ,
        -1.7705295 ],
       [-1.7308289 ,  6.9046736 , -1.3093479 , -2.0382638 , -1.6137897 ,
        -0.49515936]], dtype=float32), label_ids=array([0, 0, 2, ..., 1, 1, 1]), metrics={'test_loss': 0.15769121050834656, 'test_accuracy': 0.9405, 'test_runtime': 26.0762, 'test_samples_per_second': 76.698, 'test_steps_per_second': 19.175})

In [30]:
print("Accuracy with Standart Pretrained Model : ", predictions.metrics["test_accuracy"], " + Test Runtime : ", predictions.metrics["test_runtime"])

Accuracy with Standart Pretrained Model :  0.9405  + Test Runtime :  26.0762


In [31]:
print("Accuracy with Fine-Tuned Model : ", predictions_wPEFT.metrics["test_accuracy"], " + Test Runtime : ", predictions_wPEFT.metrics["test_runtime"])

Accuracy with Fine-Tuned Model :  0.9405  + Test Runtime :  25.6981


In [34]:
import pandas as pd

df = pd.DataFrame(
    {
        "text": [item["text"] for item in tokenized_dataset_validation["validation"]],
        "predictions": predictions.predictions.argmax(axis=1),
        "predictions w PEFT": predictions_wPEFT.predictions.argmax(axis=1),
        "labels": predictions.label_ids,
    }
)
# Show all the cell
pd.set_option("display.max_colwidth", None)
df[:10]

,text,predictions,predictions w PEFT,labels
0,im feeling quite sad and sorry for myself but ill snap out of it soon,0,0,0
1,i feel like i am still looking at a blank canvas blank pieces of paper,0,0,0
2,i feel like a faithful servant,1,1,2
3,i am just feeling cranky and blue,3,3,3
4,i can have for a treat or if i am feeling festive,1,1,1
5,i start to feel more appreciative of what god has done for me,1,1,1
6,i am feeling more confident that we will be able to take care of this baby,1,1,1
7,i feel incredibly lucky just to be able to talk to her,1,1,1
8,i feel less keen about the army every day,1,1,1
9,i feel dirty and ashamed for saying that,0,0,0
